# Deep Dreams (with Caffe)

This notebook demonstrates how to use [Caffe](http://caffe.berkeleyvision.org/) neural network framework to produce "dream" visuals shown in the [Google Research blog post](http://googleresearch.blogspot.ch/2015/06/inceptionism-going-deeper-into-neural.html).

It'll be interesting to see what imagery people are able to generate using the described technique. If you post images to Google+, Facebook, or Twitter, be sure to tag them with **#deepdream** so other researchers can check them out too.

##Dependencies
This notebook is designed to have as few dependencies as possible:
* Standard Python scientific stack: [NumPy](http://www.numpy.org/), [SciPy](http://www.scipy.org/), [PIL](http://www.pythonware.com/products/pil/), [IPython](http://ipython.org/). Those libraries can also be installed as a part of one of scientific packages for Python, such as [Anaconda](http://continuum.io/downloads) or [Canopy](https://store.enthought.com/).
* [Caffe](http://caffe.berkeleyvision.org/) deep learning framework ([installation instructions](http://caffe.berkeleyvision.org/installation.html)).
* Google [protobuf](https://developers.google.com/protocol-buffers/) library that is used for Caffe model manipulation.

In [1]:
# imports and basic notebook setup
from cStringIO import StringIO
import numpy as np
import scipy.ndimage as nd
import PIL.Image
from IPython.display import clear_output, Image, display
from google.protobuf import text_format

import caffe

import random
import os

def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

## Loading DNN model
In this notebook we are going to use a [GoogLeNet](https://github.com/BVLC/caffe/tree/master/models/bvlc_googlenet) model trained on [ImageNet](http://www.image-net.org/) dataset.
Feel free to experiment with other models from Caffe [Model Zoo](https://github.com/BVLC/caffe/wiki/Model-Zoo). One particularly interesting [model](http://places.csail.mit.edu/downloadCNN.html) was trained in [MIT Places](http://places.csail.mit.edu/) dataset. It produced many visuals from the [original blog post](http://googleresearch.blogspot.ch/2015/06/inceptionism-going-deeper-into-neural.html).

In [2]:
# Code to use the MIT Places NN
model_path = '/vagrant/models/places/' # substitute your path here
net_fn   = model_path + 'deploy_places205.protxt'
param_fn = model_path + 'googlelet_places205_train_iter_2400000.caffemodel'

# Patching model to be able to compute gradients.
# Note that you can also manually add "force_backward: true" line to "deploy.prototxt".
model = caffe.io.caffe_pb2.NetParameter()
text_format.Merge(open(net_fn).read(), model)
model.force_backward = True
open('tmp.prototxt', 'w').write(str(model))

net = caffe.Classifier('tmp.prototxt', param_fn,
                       mean = np.float32([104.0, 116.0, 122.0]), # ImageNet mean, training set dependent
                       channel_swap = (2,1,0)) # the reference model has channels in BGR order instead of RGB

# a couple of utility functions for converting to and from Caffe's input image layout
def preprocess(net, img):
    return np.float32(np.rollaxis(img, 2)[::-1]) - net.transformer.mean['data']
def deprocess(net, img):
    return np.dstack((img + net.transformer.mean['data'])[::-1])

##  Producing dreams

Making the "dream" images is very simple. Essentially it is just a gradient ascent process that tries to maximize the L2 norm of activations of a particular DNN layer. Here are a few simple tricks that we found useful for getting good images:
* offset image by a random jitter
* normalize the magnitude of gradient ascent steps
* apply ascent across multiple scales (octaves)

First we implement a basic gradient ascent step function, applying the first two tricks:

In [3]:
def make_step(net, step_size=1.5, end='inception_4c/output', jitter=32, clip=True):
    '''Basic gradient ascent step.'''

    src = net.blobs['data'] # input image is stored in Net's 'data' blob
    dst = net.blobs[end]

    ox, oy = np.random.randint(-jitter, jitter+1, 2)
    src.data[0] = np.roll(np.roll(src.data[0], ox, -1), oy, -2) # apply jitter shift
            
    net.forward(end=end)
    dst.diff[:] = dst.data  # specify the optimization objective
    net.backward(start=end)
    g = src.diff[0]
    # apply normalized ascent step to the input image
    src.data[:] += step_size/np.abs(g).mean() * g

    src.data[0] = np.roll(np.roll(src.data[0], -ox, -1), -oy, -2) # unshift image
            
    if clip:
        bias = net.transformer.mean['data']
        src.data[:] = np.clip(src.data, -bias, 255-bias)    

Next we implement an ascent through different scales. We call these scales "octaves".

In [4]:
def deepdream(net, base_img, iter_n=10, octave_n=4, octave_scale=1.4, end='inception_4c/output', clip=True, **step_params):
    # prepare base images for all octaves
    octaves = [preprocess(net, base_img)]
    for i in xrange(octave_n-1):
        octaves.append(nd.zoom(octaves[-1], (1, 1.0/octave_scale,1.0/octave_scale), order=1))
    
    src = net.blobs['data']
    detail = np.zeros_like(octaves[-1]) # allocate image for network-produced details
    for octave, octave_base in enumerate(octaves[::-1]):
        h, w = octave_base.shape[-2:]
        if octave > 0:
            # upscale details from the previous octave
            h1, w1 = detail.shape[-2:]
            detail = nd.zoom(detail, (1, 1.0*h/h1,1.0*w/w1), order=1)

        src.reshape(1,3,h,w) # resize the network's input image size
        src.data[0] = octave_base+detail
        for i in xrange(iter_n):
            make_step(net, end=end, clip=clip, **step_params)
            
            # visualization
            vis = deprocess(net, src.data[0])
            if not clip: # adjust image contrast if clipping is disabled
                vis = vis*(255.0/np.percentile(vis, 99.98))
            #showarray(vis)
            #print octave, i, end, vis.shape
            #clear_output(wait=True)
            
        # extract details produced on the current octave
        detail = src.data[0]-octave_base
    # returning the resulting image
    return deprocess(net, src.data[0])

Now we are ready to let the neural network to reveal its dreams! Let's take a [cloud image](https://commons.wikimedia.org/wiki/File:Appearance_of_sky_for_weather_forecast,_Dhaka,_Bangladesh.JPG) as a starting point:

In [5]:
#img = np.float32(PIL.Image.open('freesideDoggie.jpg'))
#showarray(img)

Running the next code cell starts the detail generation process. You may see how new patterns start to form, iteration by iteration, octave by octave.

In [6]:
#_=deepdream(net, img)

#frame_i = 0
#frame = deepdream(net, img, iter_n=20, octave_n=6)
#PIL.Image.fromarray(np.uint8(frame)).save("frames/%04d.jpg"%frame_i)

The complexity of the details generated depends on which layer's activations we try to maximize. Higher layers produce complex features, while lower ones enhance edges and textures, giving the image an impressionist feeling:

In [7]:
#_=deepdream(net, img, end='inception_3b/5x5_reduce')

We encourage readers to experiment with layer selection to see how it affects the results. Execute the next code cell to see the list of different layers. You can modify the `make_step` function to make it follow some different objective, say to select a subset of activations to maximize, or to maximize multiple layers at once. There is a huge design space to explore!

In [8]:
#  Print the types of filters that are available.
#net.blobs.keys()



In [ ]:

def letters(input):
    return ''.join([c for c in input if c.isalnum()])


def iterateEnds(imgName, frame_i, dirName, endNames, iter_n, octave_n, total_iter):
    s = 0.05 # scale coefficient
    img = np.float32(PIL.Image.open(imgName))

    for endType in endNames:
        frame = img
        h, w = frame.shape[:2]
        frame_i = 0
        try:
            for i in xrange(total_iter):
                frame = deepdream(net, frame, iter_n, octave_n, end=endType)
                PIL.Image.fromarray(np.uint8(frame)).save(dirName + "/" + letters(endType) + "%04d.jpg"%frame_i)
                frame = nd.affine_transform(frame, [1-s,1-s,1], [h*s/2,w*s/2,0], order=1)
                frame_i += 1
        except ValueError:
            print endType + " not in list"
            
            
def zoomAndProcess(img, frame_i, dirName, endNames, iter_n, octave_n, zoomsPerDream, total_iter, zoomLocation):
    s = 0.02 # scale coefficient
    frame = img
    h, w = frame.shape[:2]
    endChoice = random.choice(endNames)
    
    for i in xrange(total_iter):
        # Randomize the algorithm every few iterations
        if (i % 5 == 0): endChoice = random.choice(endNames)
        #try:
        # dream into the frame
        frame = deepdream(net, frame, iter_n, octave_n, end=endChoice)
        for i in xrange(zoomsPerDream):
            # save the new frame
            PIL.Image.fromarray(np.uint8(frame)).save(dirName + "/frame" + "%04d.jpg"%frame_i)
            # zoom in by the preset amount
            frame = nd.affine_transform(frame, [1-s,1-s,1], [h*s/zoomLocation[0],w*s/zoomLocation[1],0], order=1)
            frame_i += 1
        #except ValueError:
        #        print endChoice + " not in list"
            
    return frame, frame_i

#def createGifAndVideo(frameNames,frameDir, frameRate, outName):
#    command = "ffmpeg -r 30 -i frame%04d.jpg out.gif"


What if we feed the `deepdream` function its own output, after applying a little zoom to it? It turns out that this leads to an endless stream of impressions of the things that the network saw during training. Some patterns fire more often than others, suggestive of basins of attraction.

We will start the process from the same sky image as above, but after some iteration the original image becomes irrelevant; even random noise can be used as the starting point.

In [ ]:
# Start a new image series

# Initialize user parameters
imgName = "VideoProjectStill1.jpg"
dirName = "VideoProject3_places"
imgDir = dirName + "/" + imgName
zoomLocation = [50.0,50.0]
# For the ImageNet CNN
endNamesLight = ['inception_4a/1x1','inception_4a/5x5_reduce','conv2/3x3',
            'inception_3a/5x5','inception_3a/output','inception_3b/3x3',
            'inception_4a/pool_proj']
endNamesHeavy = ['inception_4a/pool_proj','inception_3a/pool_proj','inception_4c/output', 'inception_4d/pool_proj', 
                'inception_3b/pool_proj', 'inception_4d/1x1', 'inception_4d/3x3',
                 'inception_5a/output', 'inception_5b/output']
# For the MIT Places CNN
endNamesLight = ['inception_4a/3x3','inception_4a/output',
                 'inception_4a/pool','inception_4b/1x1',
                 'inception_4b/pool','inception_4c/5x5_reduce',
                 'inception_4c/pool','inception_4c/pool_proj',
                 'inception_4d/1x1','inception_4d/3x3',
                 'inception_4d/5x5_reduce','inception_4d/5x5',
                 'inception_4d/output','inception_4e/3x3_reduce',
                 'inception_4e/5x5_reduce','inception_4e/5x5',
                 'inception_4e/pool','inception_4e/pool_proj',
                 'inception_5a/pool','inception_5a/pool_proj',
                 'inception_5a/5x5','pool4/3x3_s2']
endNamesHeavy = ['inception_4a/5x5_reduce','inception_4a/5x5',
                 'inception_4b/3x3_reduce','inception_4b/3x3',
                 'inception_4b/5x5_reduce','inception_4b/5x5',
                 'inception_4b/pool_proj','inception_4b/output',
                 'inception_4c/3x3','inception_4d/pool',
                 'inception_4d/pool_proj','inception_4e/1x1',
                 'inception_4e/3x3','inception_4e/output',
                 'inception_5a/pool_proj','inception_5a/output',
                 'inception_5b/3x3_reduce','inception_5b/5x5_reduce',
                 'inception_5b/pool','inception_5b/pool_proj',
                 'inception_5b/output']

ends = net.blobs.keys()[1:-1]
endsInception = [e for e in ends if e[:3] == "inc"]

# initialize other parameters
frame_i = 0
zoomLocation[:] = [100.0 / x for x in zoomLocation]
if not os.path.exists(dirName):
    os.makedirs(dirName)

def startNewSeries(imgName,frame_i,dirName,endNamesLight,endNamesHeavy,zoomLocation):
    # Do Image Processing
    img = np.float32(PIL.Image.open(imgName))
    # zoomAndProcess( img, dirName, endNames, iter_n, octave_n, zoomsPerDream, total_iter, zoomLocation )
    frame, frame_i = zoomAndProcess(img,   frame_i,dirName,endNamesLight,1,1,3,12,zoomLocation)  #48
    frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesLight,2,2,2,20,zoomLocation)  #108
    frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesLight,4,3,2,30,zoomLocation)  #198
    frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesHeavy,4,3,1,50,zoomLocation)  #298
    frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesHeavy,10,4,1,80,zoomLocation) #458
    print "Done!"

# Flip through each "end" and look at the kinds of images that it produces
#iterateEnds(imgName, frame_i, dirName, ends, 10, 4, 5)

# Start a series where the zoom gradually reveals the dreams at the low level.
startNewSeries(imgName,frame_i,dirName,endNamesLight,endNamesHeavy,zoomLocation)


pool2/3x3_s2_pool2/3x3_s2_0_split_0 not in list
pool2/3x3_s2_pool2/3x3_s2_0_split_1 not in list
pool2/3x3_s2_pool2/3x3_s2_0_split_2 not in list
pool2/3x3_s2_pool2/3x3_s2_0_split_3 not in list
inception_3a/output_inception_3a/output_0_split_0 not in list
inception_3a/output_inception_3a/output_0_split_1 not in list
inception_3a/output_inception_3a/output_0_split_2 not in list

Be careful running the code above, it can bring you into very strange realms!

In [ ]:
# Continue an image series

# Initialize user parameters
dirName = "VideoProject2_zoom"
frame_i = 315  # The last good frame available
zoomLocation = [50.0,50.0]

# initialize other parameters
imgName = "frame" + "%04d.jpg"%(frame_i)
imgDir = dirName + "/" + imgName

# Pick up the series at the desired processing level
def continueSeries(imgDir,frame_i,dirName,endNamesLight,endNamesHeavy,zoomLocation):
    if not os.path.isfile(imgDir):
        print "File " + imgDir + " does not exist"
        return -1
    
    # Do Image Processing
    frame = np.float32(PIL.Image.open(imgDir))
    # zoomAndProcess( img, dirName, endNames, iter_n, octave_n, zoomsPerDream, total_iter )
    #frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesLight,1,1,4,12,zoomLocation)  #48
    #frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesLight,2,2,3,20,zoomLocation)  #108
    #frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesLight,4,3,3,30,zoomLocation)  #198
    #frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesHeavy,4,3,2,50,zoomLocation)  #298
    frame, frame_i = zoomAndProcess(frame, frame_i,dirName,endNamesHeavy,10,4,2,80,zoomLocation) #458
    return 1
    
# Run the function to continue compiling images
#continueSeries(imgDir,frame_i,dirName,endNamesLight,endNamesHeavy,zoomLocation)



# Guided Dreams

The image detail generation method described above tends to produce some patterns more often the others. One easy way to improve the generated image diversity is to tweak the optimization objective. Here we show just one of many ways to do that. Let's use one more input image. We'd call it a "guide".

Note that the neural network we use was trained on images downscaled to 224x224 size. So high resolution images might have to be downscaled, so that the network could pick up their features. The image we use here is already small enough.

Now we pick some target layer and extract guide image features.

Instead of maximizing the L2-norm of current image activations, we try to maximize the dot-products between activations of current image, and their best matching correspondences from the guide image.

This way we can affect the style of generated images without using a different training set.

In [ ]:
guide = np.float32(PIL.Image.open('flowers.jpg'))
showarray(guide)

In [ ]:
end = 'inception_3b/output'
h, w = guide.shape[:2]
src, dst = net.blobs['data'], net.blobs[end]
src.reshape(1,3,h,w)
src.data[0] = preprocess(net, guide)
net.forward(end=end)
guide_features = dst.data[0].copy()

def objective_guide(dst):
    x = dst.data[0].copy()
    y = guide_features
    ch = x.shape[0]
    x = x.reshape(ch,-1)
    y = y.reshape(ch,-1)
    A = x.T.dot(y) # compute the matrix of dot-products with guide features
    dst.diff[0].reshape(ch,-1)[:] = y[:,A.argmax(1)] # select ones that match best

_=deepdream(net, img, end=end, objective=objective_guide)